In [1]:
#Install all the packages
!conda install -y -c anaconda lxml
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.4.5~ --> anaconda::ca-certificates-2020.1.1-0
  certifi            conda-forge::certifi-2020.4.5.1-py36h~ --> anaconda::certifi-2020.4.5.1-py36_0
  openssl            conda-forge::openssl-1.1.1g-h516909a_0 --> anaconda::openssl-1.1.1g-h7b6447c_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be UPDATED:

  ca-certificates      anaconda::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2020.4.5.1-hecc5488_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi   

In [2]:
import lxml
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize
import folium
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

# **Part 1**

In [3]:
#get data from wiki page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req=pd.read_html(url)
data=req[0]

#Filter out all not assigned boroughs
df=data[data['Borough']!='Not assigned'].reset_index(drop=True)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [4]:
#get number of rows
df.shape[0]

103

# **Part 2**

In [5]:
#load in csv data
geourl='http://cocl.us/Geospatial_data'
geodata=pd.read_csv(geourl)

#Merge by intersaction of keys (default)
df2=df.merge(geodata)
df2

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# **Part 3**

In [6]:
#Filter out boroughs that only contain Toronto
df3=df2[df2['Borough'].str.contains('Toronto')].reset_index(drop=True)
df3

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [18]:
city = 'Downtown Toronto'
#get location of downtown toronto
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(city)
lat = location.latitude
lon = location.longitude

# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[lat, lon], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df3['Latitude'], df3['Longitude'], df3['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## analyse data from downtown toronto

In [8]:
#since there are more neighborhood from downtown 
df_final=df3[df3['Borough']=='Downtown Toronto'].reset_index(drop=True)
df_final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [9]:
#explore first neighborhood
reg_latitude = df_final.loc[0, 'Latitude'] # neighborhood latitude value
reg_longitude = df_final.loc[0, 'Longitude'] # neighborhood longitude value

reg_name = df_final.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(reg_name, 
                                                               reg_latitude, 
                                                               reg_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


## Get top 50 venus in Regent Park, Harbourfront

In [10]:
CLIENT_ID = '2OMIVBGRDZ4GVNN4YNWCNZUZ5VJNHRYFMJOPWS5Y5E3D53QV' 
CLIENT_SECRET = 'CLGJCNM0VJ3MFEBC1O0WYDRQ0E0EE3Z0BWMQQMMQ1QFZOPIX' 
VERSION = '20180605'

LIMIT=50
radius=500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    reg_latitude, 
    reg_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=2OMIVBGRDZ4GVNN4YNWCNZUZ5VJNHRYFMJOPWS5Y5E3D53QV&client_secret=CLGJCNM0VJ3MFEBC1O0WYDRQ0E0EE3Z0BWMQQMMQ1QFZOPIX&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=50'

In [11]:
import requests
results = requests.get(url).json()
#results have been examined but its too long so I have removed it

### convert json into df

In [12]:
#define function to get categories
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
#Make into dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
3,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
4,Body Blitz Spa East,Spa,43.654735,-79.359874
5,Impact Kitchen,Restaurant,43.656369,-79.356980
6,Corktown Common,Park,43.655618,-79.356211
7,The Extension Room,Gym / Fitness Center,43.653313,-79.359725
8,The Distillery Historic District,Historic Site,43.650244,-79.359323
9,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503


## repeat for other neighbourhoods 

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
df_venues = getNearbyVenues(names=df_final['Neighborhood'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )
df_venues.shape

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


(798, 7)

In [19]:
df_venues.groupby('Neighborhood')['Venue'].count().to_frame()

,Venue
Neighborhood,
Berczy Park,50
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15
Central Bay Street,50
Christie,17
Church and Wellesley,50
"Commerce Court, Victoria Hotel",50
"First Canadian Place, Underground city",50
"Garden District, Ryerson",50
"Harbourfront East, Union Station, Toronto Islands",50


## analyse each neighbourhood

In [20]:
# one hot encoding
onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = df_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#groupbyneighbourbood and take mean
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.02,0.0,0.00,0.0,0.0,0.02,0.0,0.0,0.0,0.00
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00
2,Central Bay Street,0.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.02,0.0,0.00,0.0,0.0,0.02,0.0,0.0,0.0,0.00
3,Christie,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00
4,Church and Wellesley,0.04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.00,0.0,0.02,0.0,0.0,0.00,0.0,0.0,0.0,0.02


## get top 5 venues within each neighbourhood

In [22]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1  Cocktail Bar  0.06
2      Beer Bar  0.04
3   Cheese Shop  0.04
4        Bakery  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3       Coffee Shop  0.07
4               Bar  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1  Italian Restaurant  0.06
2      Sandwich Place  0.06
3     Bubble Tea Shop  0.04
4        Burger Joint  0.04


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3      Nightclub  0.06
4     Baby Store  0.06


----Church and Wellesley----
              venue  freq
0        Restaurant  0.06
1       Coffee Shop  0.06
2  Sushi Restaurant  0.06
3       Yoga Studio  0.04
4         Gastropub  0.04


----Commerce Court, Victoria Hot

In [23]:
#define function to sort
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
# Convert into dataframe

import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Café,Restaurant,Seafood Restaurant,Cheese Shop,Beer Bar,Park,Liquor Store
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Coffee Shop,Boat or Ferry,Harbor / Marina,Boutique,Bar,Airport Gate
2,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Ice Cream Shop,Café,Burger Joint,Bubble Tea Shop,Dessert Shop,Department Store,Seafood Restaurant
3,Christie,Grocery Store,Café,Park,Baby Store,Diner,Coffee Shop,Nightclub,Restaurant,Italian Restaurant,Candy Store
4,Church and Wellesley,Restaurant,Coffee Shop,Sushi Restaurant,Men's Store,Japanese Restaurant,Gay Bar,Gastropub,Yoga Studio,Burrito Place,Café


## Cluster the neghbourhoods !!!

In [25]:

# set number of clusters
kclusters = 5

clustering = grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering)

In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df_final
df_merged = df_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
df_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Mexican Restaurant,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Wings Joint,Diner,Park,Music Venue,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Hobby Shop,Gym
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Café,Coffee Shop,Clothing Store,Cosmetics Shop,Tea Room,Theater,Bookstore,Ramen Restaurant,Middle Eastern Restaurant,Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Gastropub,Coffee Shop,American Restaurant,Cocktail Bar,Creperie,Cosmetics Shop,Ice Cream Shop,Bookstore,Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Bakery,Café,Restaurant,Seafood Restaurant,Cheese Shop,Beer Bar,Park,Liquor Store


In [28]:
# create map
map_clusters = folium.Map(location=[lat, lon], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## examine th clusters

In [29]:
df_merged.loc[df_merged['Cluster Labels'] == 0, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Mexican Restaurant,Restaurant
1,Downtown Toronto,0,Coffee Shop,Wings Joint,Diner,Park,Music Venue,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Hobby Shop,Gym
5,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Ice Cream Shop,Café,Burger Joint,Bubble Tea Shop,Dessert Shop,Department Store,Seafood Restaurant


In [30]:
df_merged.loc[df_merged['Cluster Labels'] == 1, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Café,Coffee Shop,Clothing Store,Cosmetics Shop,Tea Room,Theater,Bookstore,Ramen Restaurant,Middle Eastern Restaurant,Restaurant
3,Downtown Toronto,1,Café,Gastropub,Coffee Shop,American Restaurant,Cocktail Bar,Creperie,Cosmetics Shop,Ice Cream Shop,Bookstore,Restaurant
4,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Bakery,Café,Restaurant,Seafood Restaurant,Cheese Shop,Beer Bar,Park,Liquor Store
7,Downtown Toronto,1,Coffee Shop,Café,Pizza Place,Restaurant,Concert Hall,American Restaurant,Steakhouse,Gym / Fitness Center,Mediterranean Restaurant,Salon / Barbershop
8,Downtown Toronto,1,Coffee Shop,Aquarium,Park,Brewery,Plaza,Hotel,Café,Roof Deck,Basketball Stadium,Deli / Bodega
9,Downtown Toronto,1,Café,Coffee Shop,Hotel,Seafood Restaurant,Restaurant,Gastropub,Japanese Restaurant,Ice Cream Shop,IT Services,Beer Bar
10,Downtown Toronto,1,Café,Coffee Shop,Hotel,Gym,American Restaurant,Seafood Restaurant,Beer Bar,Restaurant,Deli / Bodega,Japanese Restaurant
11,Downtown Toronto,1,Café,Restaurant,Bookstore,Bar,Italian Restaurant,Japanese Restaurant,Bakery,Beer Bar,Beer Store,Sandwich Place
12,Downtown Toronto,1,Café,Bakery,Mexican Restaurant,Coffee Shop,Gaming Cafe,Vietnamese Restaurant,Dessert Shop,Vegetarian / Vegan Restaurant,Burger Joint,Belgian Restaurant
15,Downtown Toronto,1,Café,Coffee Shop,Beer Bar,Hotel,Bakery,Restaurant,Creperie,Seafood Restaurant,Cocktail Bar,Cheese Shop


In [31]:
df_merged.loc[df_merged['Cluster Labels'] == 3, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3,Park,Playground,Trail,Comic Shop,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Creperie


In [32]:
df_merged.loc[df_merged['Cluster Labels'] == 4, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,4,Grocery Store,Café,Park,Baby Store,Diner,Coffee Shop,Nightclub,Restaurant,Italian Restaurant,Candy Store


# Conclusion: Cafe/Coffee shop is most common (as in from cluster label 1)